In [44]:
# from geopy.geocoders import Nominatim
import geopy
import pandas as pd
import numpy as np
import math
import itertools
import re
import Levenshtein

In [45]:
excel = 'transmission.xlsx'
sheet = 'Trans.Investments'

df = pd.read_excel(excel, sheet_name=sheet)

# TODO: might not work in later versions, use sanity check.
df.columns = df.iloc[0]
df         = df.drop(index=df.index[0])

# TODO: also contains new substations!

wanted_columns = ['Investment number',
                  'This investment belongs to project number…',
                  'Commissioning Year',
                  'Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction',
                  'Type of Element',
                  'Substation From',
                  'Substation To',
                  'Technology',
                  'Total route length (km)']
status_column = wanted_columns[3]
df[wanted_columns]

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
1,2,1,2024,2,ACTransmissionLine,Pedralva (PT),Sobrado (PT),AC,67
2,4,1,2022,3,ACTransmissionLine,V.Minho (by Ribeira de Pena),Feira (by Ribeira de Pena),AC,131
3,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0
4,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30
5,496,4,2022,3,ACTransmissionLine,Fontefria (ES),Vila Nova de Famalicão (PT) (By Ponte de Lima),AC,140.21
...,...,...,...,...,...,...,...,...,...
317,1747,1077,2036,1,DCTransmissionLine,Thessaloniki,Dubrovo,DC,110
318,1748,1077,2036,1,DCTransmissionLine,Dubrovo,Ch. Mogila,DC,170
319,1745,1081,2036,1,OffshoreDCTransmissionCable,Tobruk,Arachtos,DC,1070
320,1749,1081,2036,1,DCTransmissionLine,Arachtos,Elbasan,DC,180


In [46]:
wanted    = df[wanted_columns]
# only choose those in permitting or under construction
wanted    = wanted.loc[wanted[status_column].astype(int) >= 3]

ac_lines  = wanted.loc[wanted['Type of Element'] == 'ACTransmissionLine']
dc_lines  = wanted.loc[wanted['Type of Element'] == 'DCTransmissionLine']
on_subst  = wanted.loc[wanted['Type of Element'] == 'OnshoreSubstation']
off_subst = wanted.loc[wanted['Type of Element'] == 'OffshoreSubstation']

# Use bus names from buses.csv (v0.1.0)

In [47]:
buses_file = 'buses_v0.1.0.csv'

# see base_network.py in PyPSA-Eur repository
buses = (pd.read_csv(buses_file, quotechar="'",
                     true_values='t', false_values='f',
                     dtype=dict(bus_id="str"))
        .set_index("bus_id")
        .drop(['station_id'], axis=1)
        .rename(columns=dict(voltage='v_nom')))

In [48]:
no_tags = buses[buses['tags'].isna()].index.size
print(f'{no_tags} buses have no tags.')

yes_tags = buses[~buses['tags'].isna()].index.size
print(f'{yes_tags} buses have tags.')

1238 buses have no tags.
6773 buses have tags.


In [49]:
buses = buses.loc[~buses['tags'].isna()]
buses = buses.loc[buses.symbol == 'Substation']

# Extract 'name_eng' and 'country' from tags in  buses

In [50]:
split_regex = r'("\w+"=>"[^"]*"),' # Form: 'key => value, key => value, ...'

tag_regex   = r'"(?P<key>\w+)"=>"(?P<value>[^"]*)"' # Form: 'key => value'
tag_pattern = re.compile(tag_regex)

rows = []

for index, row in buses.iterrows():
    name    = ''
    country = ''
    x = row['x']
    y = row['y']
    
    tags_string = row['tags']
    
    tags = re.split(split_regex, tags_string)
    
    # Remove whitespaces at front and end, remove None values
    tags = [s.strip() for s in tags]
    tags = list(filter(None, tags))
    
    for tag in tags:
        m = tag_pattern.match(tag)
            
        if m is None:
            print(tag)
            
        # see group names in tag_regex
        key   = m.group('key')
        value = m.group('value')
        
        if key == 'name_eng':
            name = value.strip()
        elif key == 'country':
            country = value.strip()
    
    if name == 'unknown' or not name:
        continue
        
    rows.append((name, country, x, y))

In [51]:
curated_buses = pd.DataFrame.from_records(rows, columns=['name', 'country', 'x', 'y'])

## Remove duplicate rows

In [52]:
curated_buses = curated_buses.loc[~curated_buses.duplicated()]

## There are substations which share the same name but have different coordinates
- large deviation between coordinates => substations are most likely in different countries 
    - BUT: it does occur that different places in the same country get the same name
- small deviation between coordinates => reference to same substation (error in gridextract?)

### List of all duplicates

In [53]:
duplicated = curated_buses.loc[curated_buses.name.duplicated()]

for name in duplicated.name.unique():
    print(name)
    for index, row in curated_buses.loc[curated_buses['name'] == name].iterrows():
        print(f"({row['x']}, {row['y']}), {row['country']}")
    print('----')

Jeddah
(39.716949, 21.985075), SA
(39.711456, 22.038549), SA
----
Al Madina
(39.635925, 24.775513), SA
(39.508209, 24.718143), SA
----
I. Baroud
(30.734253, 30.955236), EG
(30.890808, 30.855079), EG
----
Local
(25.041962, 31.598422), LY
(24.134216, 31.960318), LY
----
Sahab
(36.021423, 31.809895), JO
(36.478729, 32.261588), JO
----
Tazoult
(6.378937, 35.424868), DZ
(6.49154699999999, 35.373375), DZ
----
Saida
(35.400696, 33.587167), LB
(0.146942000000011, 34.908458), DZ
----
Tizi Ouzou
(4.15557899999999, 36.698154), DZ
(4.000397, 36.561497), DZ
----
Jerada
(-2.06405600000001, 33.876117), MA
(-2.11898799999999, 34.335498), MA
----
Taza
(-4.139099, 34.271971), MA
(-3.95919799999999, 34.268566), MA
----
Naama 400
(-0.365294999999999, 33.26625), DZ
(-0.429840000000002, 33.366091), DZ
----
Brindisi /S.
(17.992859, 40.571197), IT
(17.862396, 40.520063), IT
----
Tunis
(10.140381, 36.73118), TN
(10.055237, 36.79499), TN
----
PB 1
(7.04772899999999, 36.857648), DZ
(7.138367, 36.891703), DZ
----

### Same name and country, large deviations

In [54]:
curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']

,name,country,x,y
2720,Yuzhnaya,RU,44.817352,48.155093
3851,Yuzhnaya,RU,50.674438,52.002638
3905,Yuzhnaya,RU,36.268616,51.642737
3927,Yuzhnaya,RU,38.685608,51.843414
5378,Yuzhnaya,RU,59.824677,56.576128


### Same name, different country, large deviation

In [55]:
curated_buses.loc[curated_buses.name == 'Saida']

,name,country,x,y
617,Saida,LB,35.400696,33.587167
833,Saida,DZ,0.146942,34.908458


In [56]:
curated_buses.loc[curated_buses.name == 'Titan']

,name,country,x,y
1986,Titan,AL,19.786377,41.619549
2825,Titan,UA,33.767853,46.195993
5825,Titan,RU,34.026031,67.451763


## (TODO) Add new substations

In [57]:
on_subst

# extract country if it matches regex
# otherwise, np.NAN

,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km)
3,474,1,2021,3,OnshoreSubstation,Ribeira de Pena (PT),-,AC,0
7,499,4,2022,3,OnshoreSubstation,Beariz (ES),Beariz (ES),AC,0
8,500,4,2022,3,OnshoreSubstation,Ponte de Lima (PT),Ponte de Lima (PT),AC,0
59,715,138,2023,4,OnshoreSubstation,Stalpu (RO),Stalpu (RO),AC,0
68,701,144,2025,4,OnshoreSubstation,Resita (RO),Resita (RO),AC,0
69,705,144,2025,3,OnshoreSubstation,Timisoara (RO),Timisoara (RO),AC,0
110,1711,200,2023,4,OnshoreSubstation,Kocin (CZ),NaN,AC,0
111,1712,200,2021,4,OnshoreSubstation,Vitkov (CZ),NaN,AC,0
123,631,227,2024,3,OnshoreSubstation,Bajina Basta (RS),Bajina Basta (RS),AC,0
126,1528,227,2020,4,OnshoreSubstation,SS Kraljevo,SS Kraljevo,AC,0


## create mapping from all unique tyndp substation names to substation names from 'buses'

In [58]:
tyndp_subs   = set(ac_lines['Substation From']).union(set(ac_lines['Substation To']))
tyndp_to_bus = {}

for tyndp in tyndp_subs:
    buses_subs = curated_buses.name.values
    
    closest = min([(bus, Levenshtein.distance(bus, tyndp)) for bus in buses_subs], key=lambda t: t[1])[0]
    
    tyndp_to_bus[tyndp] = closest

In [59]:
tyndp_to_bus

{'WURMLACH': 'KIMA',
 'Chamoson': 'Chamoson',
 'Zandvliet': 'Zandvliet',
 'Liefkenshoek (BE)': 'Sbeneh N (M2)',
 'Heviz (HU) \\ Zerjavinec (HR)': 'Žerjavinec',
 'Nauders (AT)': 'Calders',
 'St. Peter': 'St. Peter',
 'Tartu': 'Tartu',
 'KHAE': 'HAM',
 'Lavorgo': 'Lavorgo',
 'Varna(BG)': 'Varna',
 'Keminmaa': 'Keminmaa',
 'Vilnius': 'Vilnius',
 'Valmiera': 'Valmiera',
 'Ganderkesee (DE)': 'Ganderkesee',
 'Visegrad (BA)': 'Visegrad',
 'Diemen': 'Diemen',
 'Elbasan': 'Elbistan',
 'Seyring': 'Seraing',
 'Glorenza (IT)': 'Glorenza',
 'Vitkov (CZ)': 'Vítkov',
 'Tuomela B': 'Tudela',
 'St. Peter (AT)': 'St. Peter',
 'Altomonte (IT)': 'Altomonte',
 'Steinach (AT)': 'Sbeneh N (M2)',
 'Colunga (IT)': 'Colunga',
 'Baczyna (PL)': 'Baczyna',
 'Endrup (DKW)': 'Endrup',
 'Baczyna': 'Baczyna',
 'Sindi (EE)': 'Sidi Ifni',
 'Rimavska\xa0Sobota (SK)': 'Rimavská Sobota',
 'SS Kragujevac': 'Kragujevac',
 'Van Eyck (BE)': 'Van Eyck',
 'Brunsb\x81üttel (DE)': 'Brunsbüttel',
 'La Punt': 'La Font',
 'Riga HPP (

### Helper functions: Out of all possible pairs of locations from two lists, get the pair whose distance is closest to the specified (line) length
Deals with problem of multiple places in same country sharing a name.

In [60]:
def extract_coords(rows):
    coordinates = []
    for index, row in rows.iterrows():
        coordinates.append((row['x'], row['y']))
    return coordinates

In [66]:
def match_pair_with_length(s1_rows, s2_rows, length):
    s1_coords = extract_coords(s1_rows)
    s2_coords = extract_coords(s2_rows)
    
    combinations  = list(itertools.product(s1_coords, s2_coords))
    with_distance = [(a, b, geopy.distance.distance(a,b).km) for (a,b) in combinations]
    
    best_match = min(with_distance, key=lambda t: abs(length - t[2]))
    return best_match

In [67]:
buses_s1 = curated_buses.loc[curated_buses['name'] == 'Yuzhnaya']
buses_s2 = curated_buses.loc[curated_buses['name'] == 'Liteynaya']

best = match_pair_with_length(buses_s1, buses_s2, 430)
best

AttributeError: module 'geopy' has no attribute 'distance'

In [323]:
subst_regex   = r'(?P<place>\w+)\s*\((?P<country>\w{2})\)' # Form: 'Glorenza (IT)'
subst_pattern = re.compile(subst_regex)

fr_to_tuples = {}
error_tuples = {}

for index, row in ac_lines.iterrows():    
    # Get closest name match based on Levenshtein distance for start- and endpoints of line
    fr = row['Substation From']
    to = row['Substation To']
    
    fr_country = None
    to_country = None
    
    fr_match = subst_pattern.match(fr)
    to_match = subst_pattern.match(to)
    
    if fr_match is not None:
        fr_country = fr_match.group('country')
    if to_match is not None:
        to_country = to_match.group('country')
    
    s1 = tyndp_to_bus[fr]
    s2 = tyndp_to_bus[to]
    
    # Extract respective rows in buses to determine coordinates
    buses_s1 = curated_buses.loc[curated_buses.name == s1]
    buses_s2 = curated_buses.loc[curated_buses.name == s2]
    
    # If we were able to extract country from name, restrict chosen rows to this country.
    if fr_country is not None:
        buses_s1 = buses_s1.loc[buses_s1['country'] == fr_country]
    if to_country is not None:
        buses_s2 = buses_s2.loc[buses_s2['country'] == fr_country]
        
    if buses_s1.empty or buses_s2.empty:
        tpl = (s1, np.NAN, np.NAN, s2, np.NAN, np.NAN, np.NAN)
        error_tuples[index] = tpl
        continue
    
    # Choose pair which matches length best
    length = row['Total route length (km)']
    (x1, y1), (x2, y2), coord_dist = match_pair_with_length(buses_s1, buses_s2, length)
        
    tpl = (s1, x1, y1, s2, x2, y2, coord_dist)
    
    # TODO: Chose a tolerance of 15km for line length. Is this appropriate?
    if not math.isclose(coord_dist, length, abs_tol=15):
        error_tuples[index] = tpl
    else:
        fr_to_tuples[index] = tpl

In [324]:
percentage = len(fr_to_tuples) / ac_lines.index.size
print(f'{percentage * 100}% of lines are probably correct.')

21.468926553672315% of lines are probably correct.


In [327]:
coordinates = pd.DataFrame(index=fr_to_tuples.keys(), data=fr_to_tuples.values(), columns=['s1', 'x1', 'y1', 's2', 'x2', 'y2', 'coord_dist'])

result = ac_lines.copy()
result = result.join(coordinates)

print('Lines where we probably found the correct coordinates:')
result.loc[~result.s1.isna()]

Lines where we probably found the correct coordinates:


,Investment number,This investment belongs to project number…,Commissioning Year,"Status ID\n1 : Under Consideration,\n2 : In Planning but not permitting,\n3 : In permitting,\n4 : Under Construction",Type of Element,Substation From,Substation To,Technology,Total route length (km),s1,x1,y1,s2,x2,y2,coord_dist
4,18,4,2022,3,ACTransmissionLine,Beariz (ES),Fontefria (ES),AC,30,Beariz,-8.242493,42.375793,Fontefria,-8.393555,42.174600,27.754446
14,90,33,2023,3,ACTransmissionLine,Calenzano (IT),Colunga (IT),AC,80,Calenzano,11.126404,43.871168,Colunga,11.284332,44.494546,70.285438
29,1062,62,2020,4,ACTransmissionLine,Riga CHP2 (LV),Riga HPP (LV),AC,15,Riga TEC 2,24.117737,56.959952,Riga TEC 2,24.117737,56.959952,0.000000
40,1488,103,2022,4,ACTransmissionLine,Diemen,Ens,AC,75,Diemen,5.067444,52.238733,Ens,5.766449,52.653062,89.908740
42,1539,103,2023,3,ACTransmissionLine,Krimpen,Geertruidenberg,AC,35,Krimpen,4.652710,51.852746,Geertruidenberg,5.055084,51.661482,49.294423
43,1540,103,2025,3,ACTransmissionLine,Eindhoven,Maasbracht,AC,50,Eindhoven,5.464325,51.421477,Maasbracht,5.910645,51.179343,56.173758
58,275,138,2024,3,ACTransmissionLine,Smardan (RO),Gutinas (RO),AC,140,Smârdan,27.913513,45.611156,Gutinaş,26.872559,46.264392,132.209509
60,800,138,2021,4,ACTransmissionLine,Varna(BG),Burgas(BG),AC,86,Varna,27.430115,43.405047,Burgas,27.360077,42.575332,82.429509
74,1004,170,2025,1,ACTransmissionLine,Sindi,Paide,AC,76,Sindi,24.689026,58.377238,Paide,25.199890,58.921664,78.902655
90,1661,170,2025,2,ACTransmissionLine,Dunowo,Zydowo Kierzkowo,AC,57.11,Dunowo,16.116943,54.132674,Żydowo Kierzkowo,16.685486,54.083562,63.134929


# Determine coordinates using geopy

In [94]:
locator = Nominatim(user_agent='esm_group')
geocode = RateLimiter(locator.geocode, min_delay_seconds=0.05)

In [95]:
x1 = []
y1 = []
x2 = []
y2 = []

error_rows = []

for index, row in ac_lines.iterrows():
    fr   = row['Substation From']
    to   = row['Substation To']
    dist = row['Total route length (km)']

    fr_loc = geocode(fr)
    to_loc = geocode(to)

    if fr_loc is None or to_loc is None:
        error_rows.append([row.values])
        continue
        
    fr_coords  = fr_loc.latitude, fr_loc.longitude
    to_coords  = to_loc.latitude, to_loc.longitude
    coord_dist = distance.distance(fr_coords, to_coords).km

    if math.isclose(coord_dist, dist, rel_tol=0.25):
        x1.append(fr_coords[0])
        y1.append(fr_coords[1])
        
        x2.append(to_coords[0])
        y2.append(to_coords[1])
    else:
        error_rows.append([row.values])
        # x1.append(np.NAN)
        # y1.append(np.NAN)
        
        # x2.append(np.NAN)
        # y2.append(np.NAN)

In [99]:
len(x1), len(error_rows)

(55, 122)

In [92]:
error_rows

[array([4, 1, '2022', 3, 'ACTransmissionLine',
        'V.Minho (by Ribeira de Pena)', 'Feira (by Ribeira de Pena)', 'AC',
        131, nan, nan, nan, nan, nan], dtype=object),
 array([496, 4, '2022', 3, 'ACTransmissionLine', 'Fontefria (ES)',
        'Vila Nova de Famalicão (PT) (By Ponte de Lima)', 'AC', 140.21,
        nan, nan, nan, nan, nan], dtype=object),
 array([60, 23, '2022', 4, 'ACTransmissionLine', 'Avelin/Mastaing (FR)',
        'Horta (BE)', 'AC', 80, nan, nan, nan, nan, nan], dtype=object)]